In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
import ipywidgets as widgets
from IPython.display import display, clear_output
import datetime

# For reproducibility
np.random.seed(42)

In [5]:
def generate_data():
    dates = pd.date_range(start="2024-01-01", end="2024-12-31 23:00:00", freq='h')
    df = pd.DataFrame({'datetime': dates})

    # Features
    df['hour'] = df['datetime'].dt.hour
    df['day_type'] = df['datetime'].dt.dayofweek.apply(lambda x: 'Weekend' if x >= 5 else 'Weekday')

    # Simulate Events (e.g., random matches on Tuesday/Friday/Saturday)
    df['is_event'] = 0
    event_days = df[df['datetime'].dt.dayofweek.isin([1, 4, 5])].sample(frac=0.3).index
    df.loc[event_days, 'is_event'] = 1

    # Base usage + Day cycle + Event spikes
    # Base usage: 50-100 kWh
    # Night events (7PM - 11PM): add 300-500 kWh
    df['usage'] = 50 + 20 * np.sin(2 * np.pi * df['hour'] / 24) + np.random.normal(0, 5, len(df))
    df.loc[(df['is_event'] == 1) & (df['hour'].between(18, 23)), 'usage'] += np.random.uniform(200, 400)

    return df

df = generate_data()
df.set_index('datetime', inplace=True)
print(df.head())

                     hour day_type  is_event      usage
datetime                                               
2024-01-01 00:00:00     0  Weekday         0  45.959032
2024-01-01 01:00:00     1  Weekday         0  55.947967
2024-01-01 02:00:00     2  Weekday         0  60.567345
2024-01-01 03:00:00     3  Weekday         0  62.588363
2024-01-01 04:00:00     4  Weekday         0  69.892513


In [6]:
# Scaling
scaler = MinMaxScaler()
scaled_data = scaler.fit_transform(df[['usage', 'is_event']])

# Create Windows
def create_sequences(data, window_size=24):
    X, y = [], []
    for i in range(len(data) - window_size):
        X.append(data[i:i + window_size])
        y.append(data[i + window_size, 0]) # Predicting 'usage'
    return np.array(X), np.array(y)

WINDOW_SIZE = 24
X, y = create_sequences(scaled_data, WINDOW_SIZE)

# Split (80% Train, 20% Test)
split = int(0.8 * len(X))
X_train, X_test = X[:split], X[split:]
y_train, y_test = y[:split], y[split:]

In [7]:
model = Sequential([
    LSTM(50, activation='relu', input_shape=(WINDOW_SIZE, 2), return_sequences=False),
    Dropout(0.2),
    Dense(1)
])

model.compile(optimizer='adam', loss='mse')
print("Training model... (This may take a minute)")
history = model.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.1, verbose=1)

/usr/local/lib/python3.12/dist-packages/keras/src/layers/rnn/rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Training model... (This may take a minute)
Epoch 1/10
198/198 ━━━━━━━━━━━━━━━━━━━━ 5s 13ms/step - loss: 0.0279 - val_loss: 0.0230
Epoch 2/10
198/198 ━━━━━━━━━━━━━━━━━━━━ 2s 12ms/step - loss: 0.0193 - val_loss: 0.0231
Epoch 3/10
198/198 ━━━━━━━━━━━━━━━━━━━━ 2s 12ms/step - loss: 0.0227 - val_loss: 0.0228
Epoch 4/10
198/198 ━━━━━━━━━━━━━━━━━━━━ 3s 14ms/step - loss: 0.0218 - val_loss: 0.0241
Epoch 5/10
198/198 ━━━━━━━━━━━━━━━━━━━━ 5s 12ms/step - loss: 0.0207 - val_loss: 0.0224
Epoch 6/10
198/198 ━━━━━━━━━━━━━━━━━━━━ 2s 12ms/step - loss: 0.0222 - val_loss: 0.0226
Epoch 7/10
198/198 ━━━━━━━━━━━━━━━━━━━━ 3s 15ms/step - loss: 0.0228 - val_loss: 0.0223
Epoch 8/10
198/198 ━━━━━━━━━━━━━━━━━━━━ 6s 28ms/step - loss: 0.0215 - val_loss: 0.0226
Epoch 9/10
198/198 ━━━━━━━━━━━━━━━━━━━━ 2s 12ms/step - loss: 0.0199 - val_loss: 0.0222
Epoch 10/10
198/198 ━━━━━━━━━━━━━━━━━━━━ 2s 12ms/step - loss: 0.0219 - val_loss: 0.0217


In [ ]:
# Predictions
predictions = model.predict(X_test)

# Rescale back to original values
# Note: To inverse scale only 'usage', we create a dummy array
dummy = np.zeros((len(predictions), 2))
dummy[:, 0] = predictions.flatten()
inv_predictions = scaler.inverse_transform(dummy)[:, 0]

dummy_y = np.zeros((len(y_test), 2))
dummy_y[:, 0] = y_test
inv_actual = scaler.inverse_transform(dummy_y)[:, 0]

# Prepare results for dashboard
results_df = df.iloc[split + WINDOW_SIZE:].copy()
results_df['Actual'] = inv_actual
results_df['Predicted'] = inv_predictions



def update_dashboard(change):
    with output:
        clear_output(wait=True)
        filtered_df = results_df.copy()
        if change['new'] != 'All':
            filtered_df = results_df[results_df['day_type'] == change['new']]

        plt.figure(figsize=(12, 5))
        plt.plot(filtered_df.index[:168], filtered_df['Actual'][:168], label='Actual Usage', alpha=0.7)
        plt.plot(filtered_df.index[:168], filtered_df['Predicted'][:168], label='LSTM Prediction', linestyle='--')
        plt.title(f"Post-Event Electricity Prediction ({change['new']}) - Sample Week")
        plt.ylabel("kWh")
        plt.legend()
        plt.grid(True)
        plt.show()

dropdown.observe(update_dashboard, names='value')
display(dropdown, output)

# Initialize with 'All'
update_dashboard({'new': 'All'})

55/55 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step


Dropdown(description='Day Type:', options=('All', 'Weekday', 'Weekend'), value='All')

Output()